<a href="https://colab.research.google.com/github/Kevin1732/3d-pose-warping/blob/master/Copy_of_3D_pose_warping_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset exploration


*   Explore the **In-shop Clothes Retrieval Benchmark** of the **DeepFashion** dataset
http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion/InShopRetrieval.html
*   Import the dataset in `Google Colab` instead of downloading the whole dataset locally
*   Visualize some samples from the dataset
*   Look for variations and correlations in the dataset
*   Use the `matplotlib` and `seaborn` library to visualize data



In [ ]:
# Add code cells in this section

# ResNET Architecture



*   Read about **ResNET** architecture and advantage of residual blocks
*   Build a simple 6-layer ConvNET and train it on **CIFAR-10** dataset
*   Introduce residual blocks in the above model and compare the results 
*   Use the `tensorflow` library to import ResNET model with imgaenet weights
*   Change some parameters and document the results
*   Visualize the output of some layers and add/delete layers to see the effect of individual layers on the model accuracy



In [4]:
# I am trying using keras 
# but taking lot of time approx 4 min per epoch

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

Epoch 1/10
  41/1563 [..............................] - ETA: 4:51 - loss: 2.3055 - accuracy: 0.1220

KeyboardInterrupt: ignored

In [ ]:
# Add code cells in this section
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from keras.datasets import cifar10
#from tensorflow.examples.tutorials.mnist import input_data
# load dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, w5, w6, w_o, p_keep_conv, p_keep_hidden):
    l1 = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    #l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 28, 28, 32)
                        #strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 28, 28, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 14, 14, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3 = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    #l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 7, 7, 128)
                        #strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4a = tf.nn.relu(tf.nn.conv2d(l3, w4,                     # l4a shape=(?, 7, 7, 256)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l4 = tf.nn.max_pool(l4a, ksize=[1, 2, 2, 1],              # l4 shape=(?, 4, 4, 256)
                        strides=[1, 2, 2, 1], padding='SAME')
    l4 = tf.nn.dropout(l4, p_keep_conv)

    l5 = tf.nn.relu(tf.nn.conv2d(l4, w5,                     # l5a shape=(?, 4, 4, 512)
                        strides=[1, 1, 1, 1], padding='SAME'))
    #l5 = tf.nn.max_pool(l5a, ksize=[1, 2, 2, 1],              # l5 shape=(?, 4, 4, 512)
                        #strides=[1, 2, 2, 1], padding='SAME')
    l5 = tf.nn.dropout(l5, p_keep_conv)



    l5 = tf.reshape(l5, [-1, w6.get_shape().as_list()[0]])    # reshape to (?, 512*4*4)
    l5 = tf.nn.dropout(l5, p_keep_conv)

    l6 = tf.nn.relu(tf.matmul(l5, w6))
    l6 = tf.nn.dropout(l6, p_keep_hidden)

    pyx = tf.matmul(l6, w_o)
    return pyx

# load dataset
(trainX_c, trainY), (testX_c, testY) = cifar10.load_data()

#trainX = np.mean(trainX_c, axis=2)
#testX = np.mean(testX_c, axis=2)
# Normalize pixel values to be between 0 and 1
trainX, testX = trainX / 255.0, testX / 255.0

X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([3, 3, 128, 256])   # 3x3x32 conv, 256 outputs
w5 = init_weights([3, 3, 256, 512])   # 3x3x32 conv, 512 outputs
w6 = init_weights([512 * 4 * 4, 625]) # FC 512 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w5, w6, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)


# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        training_batch = zip(range(0, len(trainX), batch_size),
                             range(batch_size, len(trainX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trainX[start:end], Y: trainY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(testX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(testY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: testX[test_indices],
                                                         Y: testY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))

ValueError: ignored